<a href="https://colab.research.google.com/github/sharadnatarajSJSU/CMPE-255-Approximate-nearest-neighbour/blob/main/CMPE_255_ANN_SOTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle


In [4]:
!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [7]:
pip install --upgrade faiss-gpu==1.7.1

     |████████████████████████████████| 89.7 MB 15 kB/s 
  Attempting uninstall: faiss-gpu
    Found existing installation: faiss-gpu 1.7.1.post2
    Uninstalling faiss-gpu-1.7.1.post2:
      Successfully uninstalled faiss-gpu-1.7.1.post2


In [8]:
import faiss

The data set movies.pickle was created by combining labels and vectors of each movie using LightFm model.  https://colab.research.google.com/drive/1D6lbgua72MaOMtEKDO6N0EemU6TNLa99#scrollTo=abjzFcJcUadY

In [9]:
def load_data():
    with open('movies.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

In [11]:
data = load_data()
vectors = data["vector"]
names = data["name"]
data

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[-0.01780608, -0.14265831,  0.10308606, ...,  0.09659795,
         -0.17529577, -0.03061521],
        [-0.03357764,  0.16418771,  0.21801303, ...,  0.16502103,
         -0.09166156,  0.05047869],
        [-0.2761452 , -0.01991325, -0.04969981, ...,  0.0258275 ,
         -0.08328608, -0.0152858 ],
        ...,
        [ 0.05142734, -0.01683608, -0.20441587, ...,  0.00045828,
          0.14679626,  0.2462584 ],
        [ 0.04491899, -0.02819411, -0.09472758, ..., -0.02152078,
          0.16223577,  0.19897607],
        [ 0.02531924,  0.03099714,  0.06437534, ..., -0.07260127,
          0.0467432 ,  0.07893164]], dtype=float32)}

#Exhaustive searach

In [13]:
faiss.MatrixStats(vectors).comments.split("\n")

['analyzing 1682 vectors of size 64',
 'no NaN or Infs in data',
 'all vectors are distinct',
 'range of L2 norms=[0.747558, 1.80436] (0 null vectors)',
 'matrix contains no 0s',
 'no constant dimensions',
 'no dimension has a too large mean',
 'stddevs per dimension are in [0.112036 0.158214]',
 '']

Some statistics regarding the created data set

Using Faiss IndexFlatL2  to search indexes of the movie dataset to create a queryable index.

In [14]:
movie_index = faiss.IndexFlatL2(vectors.shape[1])
movie_index.add(vectors)

In [15]:
movie_index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7f493dee5b40> >

In [16]:
distances, indices = movie_index.search(vectors[55:56], 10)

In [17]:
print(f"The most similar movies to {names[55]} are:\n")
print([names[i] for i in indices[0]])

The most similar movies to Pulp Fiction (1994) are:

['Pulp Fiction (1994)', 'Silence of the Lambs, The (1991)', 'Fugitive, The (1993)', 'Clockwork Orange, A (1971)', 'Indiana Jones and the Last Crusade (1989)', 'Seven (Se7en) (1995)', 'Raiders of the Lost Ark (1981)', 'Raising Arizona (1987)', 'Terminator, The (1984)', 'Usual Suspects, The (1995)']


In [18]:
class getindex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [19]:
data_index = getindex(data["vector"], data["name"])

In [20]:
data_index.build()

In [21]:
data_index.query(data['vector'])

['Toy Story (1995)',
 'Rock, The (1996)',
 'Return of the Jedi (1983)',
 'Willy Wonka and the Chocolate Factory (1971)',
 'Phenomenon (1996)',
 'Star Trek: First Contact (1996)',
 'Star Wars (1977)',
 'Hunchback of Notre Dame, The (1996)',
 'Birdcage, The (1996)',
 'Mars Attacks! (1996)']

#Product quantization

In [22]:
product_quantizer = faiss.IndexFlatL2(vectors.shape[1])

100 8 and 8 represent the number of partitions, the search radius and vector size

In [25]:
pq_index = faiss.IndexIVFPQ(product_quantizer, vectors.shape[1],100,8,8)

In [26]:
pq_index.train(vectors)
pq_index.add(vectors)

In [30]:
distances, indices = index.search(vectors[0:1], 10)

In [31]:
print([names[i] for i in indices[0]])

['Scream of Stone (Schrei aus Stein) (1991)', 'Scream of Stone (Schrei aus Stein) (1991)', 'Scream of Stone (Schrei aus Stein) (1991)', 'Scream of Stone (Schrei aus Stein) (1991)', 'Scream of Stone (Schrei aus Stein) (1991)', 'Scream of Stone (Schrei aus Stein) (1991)', 'Scream of Stone (Schrei aus Stein) (1991)', 'Scream of Stone (Schrei aus Stein) (1991)', 'Scream of Stone (Schrei aus Stein) (1991)', 'Scream of Stone (Schrei aus Stein) (1991)']


We will build a queryable index class in the same way as Exhaustive search

In [45]:
class IVPQIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
    def build(self,number_of_partition=8,search_in_x_partitions=2,subvector_size=8):
            quantizer = faiss.IndexFlatL2(self.dimension)
            self.index = faiss.IndexIVFPQ(quantizer, self.dimension,number_of_partition,search_in_x_partitions,subvector_size)
            self.index.train(self.vectors)
            self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [41]:
pq_index = IVPQIndex(data["vector"], data["name"])
pq_index.build()

In [46]:
pq_index.query(data['vector'][0:1])

['Toy Story (1995)',
 'Fifth Element, The (1997)',
 'Men in Black (1997)',
 'James and the Giant Peach (1996)',
 'Phenomenon (1996)',
 'Hunchback of Notre Dame, The (1996)',
 'Star Trek: First Contact (1996)',
 'Twister (1996)',
 'Willy Wonka and the Chocolate Factory (1971)',
 "My Best Friend's Wedding (1997)"]

#Vector encoding using trees and graphs

In [54]:
!pip install annoy

     |████████████████████████████████| 646 kB 9.1 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391668 sha256=04d1145600ef0ff6f47c0f6029f7ec0993803cb82514659b11fff14ea9b1a442
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [57]:
import annoy

In [63]:
class Annoyindex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimension)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(
              vector.tolist(), 
              k, 
              search_k=10)                                           
        return [self.labels[i] for i in indices]

In [64]:
annoy_index = Annoyindex(data["vector"], data["name"])
annoy_index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  


In [65]:
annoy_index.query(data['vector'][0])

['Toy Story (1995)',
 'Rock, The (1996)',
 'Return of the Jedi (1983)',
 'Star Wars (1977)',
 'Willy Wonka and the Chocolate Factory (1971)',
 'Star Trek: First Contact (1996)',
 'Fargo (1996)',
 'Twelve Monkeys (1995)',
 'Mission: Impossible (1996)',
 'Hunchback of Notre Dame, The (1996)']

#HNSW

Hierarchical Navigable Small World Graphs builds graphs based on cosine similarity scores. In our case the similarity score is cosine similarity

In [70]:
!pip install nmslib

     |████████████████████████████████| 13.5 MB 9.4 MB/s 
     |████████████████████████████████| 188 kB 52.0 MB/s 


In [71]:
import nmslib

In [67]:
class HSNW_index():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [72]:
hnsw = HSNW_index(data["vector"], data["name"])
hnsw.build()

In [73]:
hnsw.query(data['vector'][0])

['Toy Story (1995)',
 'Rock, The (1996)',
 'Return of the Jedi (1983)',
 'Star Wars (1977)',
 'Willy Wonka and the Chocolate Factory (1971)',
 'Star Trek: First Contact (1996)',
 'Fargo (1996)',
 'Twelve Monkeys (1995)',
 'Phenomenon (1996)',
 'Men in Black (1997)']

#LSH

Using LSH to create a hash table that will have

In [75]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [76]:
lsh_index = LSHIndex(data["vector"], data["name"])
lsh_index.build()

In [78]:
lsh_index.query(data['vector'])

['Donnie Brasco (1997)',
 'Rumble in the Bronx (1995)',
 'Mission: Impossible (1996)',
 'Four Rooms (1995)',
 'Cold Comfort Farm (1995)',
 'Supercop (1992)',
 'Toy Story (1995)',
 'Twelve Monkeys (1995)',
 'Twister (1996)',
 'Angels and Insects (1995)']